In [1]:
import networkx
import matplotlib.pyplot as plt
from visualisation import visualize_sample

unprepped_data = "gnnet-ch23-dataset-cbr-mb"
# 0 bis 42xx
sample_info = visualize_sample(2100, dataset = "data\\" + unprepped_data)


# Infos about the sample, that are used for training
sample = sample_info.Sample.iloc[0]
# Graphs on layer 3 and layer 2
L3g = sample_info.L3_graph.iloc[0]
L2g = sample_info.L2_graph.iloc[0]

# Routing on layer 3 and layer 3
L3r = sample_info.L3_routing.iloc[0]
L2r = sample_info.L2_routing.iloc[0]

# Describes where the traffic comes from. Form <src> <dest> <traffic generator src> traffic generator dest>
tg = sample_info.tg.iloc[0]

# G2 = networkx.read_gml("data\\gnnet-ch23-dataset-cbr-mb\\graphs\\" + L2g, destringizer=int)
# G3 = networkx.read_gml("data\\gnnet-ch23-dataset-cbr-mb\\graphs\\" + L3g, destringizer=int)

G2 = networkx.read_gml("data\\" + unprepped_data + "\\graphs\\" + L2g, destringizer=int)
G3 = networkx.read_gml("data\\" + unprepped_data + "\\graphs\\" + L3g, destringizer=int)

for node2,node3 in zip(G2.nodes(data=True),G3.nodes(data=True)):
    node2[1]['scheduling_policy']= node3[1]["schedulingPolicy"]

print(L3r)
print(L2r)
print(tg)

routes = []
with open("data\\" + unprepped_data + "\\routings\\" + L2r, 'r') as f:
    for i,line in enumerate(f.readlines()):
        if i%5 == 0:
            routes.append(line.strip("\n").split(" "))
        elif len(routes) > 2:
            break


for i, route in enumerate(routes):
    for edge in G2.edges(data=True):
        if edge[2]["port"] in route:
            edge[2]["used_for"] = i+1

for edge in G2.edges(data=True):
    if "used_for" not in edge[2].keys():
        edge[2]["used_for"] = 0

Routing-topo-7-24-0-SP-k-5.txt
l2_routing-topo-7-24-0.txt
tg_per_path-topo-7-24-0.txt


In [2]:
from yfiles_jupyter_graphs import GraphWidget

w = GraphWidget(graph = G2)


def c_node_label_mapping(index, node):
    properties = node.get('properties', {})
    return str(properties.get("type"))

def c_color_mapping(index, node):
    properties = node.get('properties', {})
    router_type = str(properties.get("type"))
    color = {"r":"red",
             "s":"blue",
             "tc":"green",
             "ts":"grey"}
    return color[router_type]

def custom_edge_color_mapping(edge):
    is_used = edge["properties"]["used_for"]
    color = {0:"#d3d3d3",
             1:"red",
             2:"#eb710e",
             3:"#9aeb0e"
             }
    
    return color[is_used]


w.set_edge_color_mapping(custom_edge_color_mapping)
w.node_label_mapping = c_node_label_mapping
w.node_color_mapping = c_color_mapping

w.show()

# Visualises the graph, with routers switches and traffic generators. Highlights 3 sample routings for the l2 layer

GraphWidget(layout=Layout(height='630px', width='100%'))